**How to Display a Progress Bar while a Thread is Running in Tkinter**


This tutorial assumes that you know how to use the after() method and understand how threadings work in Python. Also, you should know how to switch between frames using the tkraise() method.

In this tutorial, you’ll build a picture viewer that shows a random picture from unsplash.com using its API.

If you make an HTTP request to the following API endpoint:

`https://source.unsplash.com/random/640x480`

…you’ll get a random picture with the size of 640×480.

When you click the Next Picture button, the program calls the API from unsplash.com to download a random picture and displays it on the window.

It’ll also show a progress bar while the picture is downloading, indicating that the download is in progress: